## Bibliotecas

In [1]:
import pandas as pd
import spacy as sp

## Carregando os dados

In [2]:
dados_treino = pd.read_csv('Dados/treino.csv')
dados_teste = pd.read_csv('Dados/teste.csv')

## Pré-processamento com Spacy

In [3]:
# Carregando o modelo de lingua portuguesa.
nlp = sp.load('pt_core_news_sm')

In [4]:
# Criando um gerador de textos para tratamento.
textos_para_tratamento = (titulos.lower() for titulos in dados_treino['title'])

In [5]:
# Criando uma função para tratamento de texto.
def tratatamento_texto(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha # Token é válido se não for stopword e for alfabético
        if e_valido: # Se for token válido, adiciona na lista de tokens válidos.
            tokens_validos.append(token.text)

    if len(tokens_validos) > 2: # Se o texto tiver mais de 2 tokens válidos, retorna o texto tratado.
        return ' '.join(tokens_validos)

In [6]:
texto = 'Um texto qualquer para ser tratado.'
doc = nlp(texto)

# Criando uma lista de textos tratados.
textos_tratados = [tratatamento_texto(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                                batch_size=1000,
                                                                n_process=-1)]

In [7]:
# Criando um dataframe com os textos tratados.
titulos_tratados = pd.DataFrame({'titulo': textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


## Criando o modelo Word2Vec

In [8]:
from gensim.models import Word2Vec

# Criando um modelo Word2Vec
w2v_modelo = Word2Vec(sg=0, # CBOW
                      window=2, # Considerando 2 palavras antes e depois.
                      vector_size=300, # Tamanho do vetor do Word2Vec
                      min_count=5, # Considerar apenas palavras que aparecem 5 vezes ou mais. 
                      alpha=0.03, # Taxa de aprendizado.
                      min_alpha=0.007) # Taxa de aprendizado (minima)


### Construindo o vocabulário do modelo

In [9]:
# Imprimindo o tamanho do vocabulário antes do tratamento.
print(len(titulos_tratados))

# Removendo linhas com valores nulos e duplicados.
titulos_tratados = titulos_tratados.dropna().drop_duplicates()

# Imprimindo o tamanho do vocabulário após o tratamento.
print(len(titulos_tratados))

# Criando uma lista de listas de tokens.
lista_lista_tokens = [titulo.split(' ') for titulo in titulos_tratados['titulo']]

90000
84466


In [10]:
import logging

# Configurando o log do modelo Word2Vec.
logging.basicConfig(format='%(asctime)s : - %(message)s', level=logging.INFO)

# Criando um modelo Word2Vec
w2v_modelo = Word2Vec(sg=0, # CBOW
                      window=2, # Considerando 2 palavras antes e depois.
                      vector_size=300, # Tamanho do vetor do Word2Vec
                      min_count=5, # Considerar apenas palavras que aparecem 5 vezes ou mais. 
                      alpha=0.03, # Taxa de aprendizado.
                      min_alpha=0.007) # Taxa de aprendizado (minima)


# Criando o vocabulário do modelo Word2Vec.
w2v_modelo.build_vocab(lista_lista_tokens, # Lista de lista de tokens de nossos títulos.
                       progress_per=5000) # A cada 5000 títulos, imprime uma mensagem de progresso.

2024-01-17 20:37:58,421 : - Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2024-01-17T20:37:58.421768', 'gensim': '4.3.0', 'python': '3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2024-01-17 20:37:58,422 : - collecting all words and their counts
2024-01-17 20:37:58,423 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-01-17 20:37:58,432 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2024-01-17 20:37:58,448 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2024-01-17 20:37:58,464 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2024-01-17 20:37:58,482 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2024-01-17 20:37:58,502 : - PROGRESS: at sentence #25000, processed 159

### Treinamento CBOW

In [11]:
# Treinando o modelo Word2Vec.
w2v_modelo.train(lista_lista_tokens, # Lista de lista de tokens de nossos títulos.
                 total_examples=w2v_modelo.corpus_count, # Número total de exemplos.
                 epochs=30) # Número de épocas de treinamento.)

2024-01-17 20:37:59,103 : - Word2Vec lifecycle event {'msg': 'training model with 3 workers on 12924 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2 shrink_windows=True', 'datetime': '2024-01-17T20:37:59.103546', 'gensim': '4.3.0', 'python': '3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'train'}
2024-01-17 20:38:00,076 : - EPOCH 0: training on 540242 raw words (485990 effective words) took 0.9s, 525745 effective words/s
2024-01-17 20:38:00,872 : - EPOCH 1: training on 540242 raw words (486141 effective words) took 0.8s, 646758 effective words/s
2024-01-17 20:38:01,588 : - EPOCH 2: training on 540242 raw words (486137 effective words) took 0.7s, 690929 effective words/s
2024-01-17 20:38:02,353 : - EPOCH 3: training on 540242 raw words (486260 effective words) took 0.8s, 645786 effective words/s
2024-01-17 20:38:03,030 : - EPOCH 4: training on 540242 raw wo

(14583855, 16207260)

### Treinamento Skip-gram

In [12]:
# Criando um modelo Word2Vec - Skip-gram
w2v_modelo_sg = Word2Vec(sg=1, # Skip-gram
                      window=5, # Considerando 5 palavras antes e depois.
                      vector_size=300, # Tamanho do vetor do Word2Vec
                      min_count=5, # Considerar apenas palavras que aparecem 5 vezes ou mais. 
                      alpha=0.03, # Taxa de aprendizado.
                      min_alpha=0.007) # Taxa de aprendizado (minima)


# Criando o vocabulário do modelo Word2Vec.
w2v_modelo_sg.build_vocab(lista_lista_tokens, # Lista de lista de tokens de nossos títulos.
                       progress_per=5000) # A cada 5000 títulos, imprime uma mensagem de progresso.

# Treinando o modelo Word2Vec.
w2v_modelo_sg.train(lista_lista_tokens, # Lista de lista de tokens de nossos títulos.
                 total_examples=w2v_modelo_sg.corpus_count, # Número total de exemplos.
                 epochs=30) # Número de épocas de treinamento.)

2024-01-17 20:38:20,222 : - Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2024-01-17T20:38:20.222700', 'gensim': '4.3.0', 'python': '3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2024-01-17 20:38:20,224 : - collecting all words and their counts
2024-01-17 20:38:20,225 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-01-17 20:38:20,242 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2024-01-17 20:38:20,258 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2024-01-17 20:38:20,267 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2024-01-17 20:38:20,276 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2024-01-17 20:38:20,286 : - PROGRESS: at sentence #25000, processed 159

(14584689, 16207260)

## Salvando os modelos

In [13]:
w2v_modelo.wv.save_word2vec_format('Dados/modelo_cbow.txt', binary=False)
w2v_modelo_sg.wv.save_word2vec_format('Dados/modelo_skipgram.txt', binary=False)

2024-01-17 20:39:16,252 : - storing 12924x300 projection weights into modelo_cbow.txt
2024-01-17 20:39:20,841 : - storing 12924x300 projection weights into modelo_skipgram.txt
